In [ ]:
# Import the necessary modules.
import numpy as np
import matplotlib.pyplot as plt

# Image processing utilities
import skimage.io
import skimage.morphology
import skimage.filters

%matplotlib inline

In this script, we'll learn some basic facts about images, how to read them
in Python, and the basic principles of thresholding. On the course website, you will find a large image set
that we will use for the whole project. Before we begin, let's talk a bit
about the nature of the image set.

These images are of *E. coli* cells with a variety of different copy numbers
of the LacI repressor molecule. This set is composed of three different LacO
sequences (O1, O2, and O3), and a variety of different repressor copy numbers
(indicated by the `R` in the image file name).  In these strains (and the
`wt` strain), the LacI repressor molecule represses the expression of a
Yellow Fluorescent Protein molecule. With more repressor around, less YFP
molecules are made. There are three strains without an `R` label. These are
`auto` which is expressing no YFP at all, `delta` which is constitutively
expressing YFP (has no repressors), and `wt` which has the wild-type number
of LacI repressors, 22 per cell.

In our project, we will quantify the fold-change in gene expression under
different repressor copy numbers. To do so, we will need to make measurements
of the single-cell fluorescence intensities in our images. We'll start by
learning about images and how to process them in the Python programming
language.

It is important to remember that an image is nothing but data -- it is an
array of points with a specific value. These points are called 'pixels'. The
values that these pixels can take is related to the construction of the
camera and is measured as 'bit depth'. To determine the range of pixel
values in an $N$ bit image ca take, we simply need to compute $2^N - 1$. This
subtraction of 1 is because 0 can be a pixel value as well. For example,
a 16-bit image can have pixels on the range of $0 \rightarrow (2^16 -1 ) = 0 \rightarrow 65535$.
Let's begin by loading an example image into Python.

In [ ]:
# load an image
dir_prefix = '/Users/muir/datasets/2017cshl_pboc/lacI_titration'
image = skimage.io.imread(dir_prefix + '/O2_delta_phase_pos_16.tif')
_, ax = plt.subplots(figsize=(10,10))
ax.imshow(image, cmap=plt.cm.Greys_r)

We see that bacterial cells are black against a light colored background.
We can see
the pixels within the bacerium are lower than those of the background. We
could select only the pixels of the bacterium by drawing a threshold at some
value and saying anything below that value is 'bacterial'. To figure out
what this threshold should be, we can look at the image histogram and pick a
value.

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.hist(image.flatten(), bins=300)
ax.set_yscale('log')
ax.set_xlabel('pixel value')
ax.set_ylabel('frequency')
ax.grid(True)

We can clearly see two humps in this image. The left most hump contains the pixels
within our bacterial cells while the major hump are the pixels of the
background. One could imagine selecting only the bacteria in this image
by choosing a threshold value between these two peaks and identifying any
pixel below this threshold as bacterial. Looking at the histogram, we can
choose a threshold of around 2500 counts.

In [ ]:
im_thresh = image < 2500
_, ax = plt.subplots(figsize=(10,10))
ax.imshow(im_thresh, cmap=plt.cm.Greys_r)

This seems to do a pretty good job at separating the cells from the
background, but it is really dependent on the actual values recorded by the
camera. These values can vary from sample to sample in an experiment. However, the ratio of the cell interior to the background should be
much less variable. Let's convert this image to a float (values ranging
between 0 and 1), choose a new threshold, and replot the new mask.

In [ ]:
# normalize image
im_float = (image - image.min()) / (image.max() - image.min())
# plot histogram of normalized intensities
_, ax = plt.subplots(figsize=(12,6))
ax.hist(im_float.flatten(), bins=300)
ax.vlines(0.28, 1e0, 3e3)
ax.set_xlabel('normalized pixel count')
ax.set_ylabel('counts')
ax.set_yscale('log')

im_float_thresh = im_float < 0.28
_, ax2 = plt.subplots(figsize=(10,10))
ax2.imshow(im_float_thresh, cmap=plt.cm.Greys_r)

Why are we getting so much of the background in our segmentation? This is
because the illumination of the image is not uniform. We can see that the
lefthand side of the image is darker than that on the right. We can easily
correct for that by performing a background subtraction. To do so, we will
very heavily blur the image and subtract it from the original. This will
remove any large scale aberrations in the intensity of the image leaving
small scale features (such as bacteria) alone. Let's go a head and give it
a shot.

In [ ]:
# carry out background subtraction
blur_radius = 50.0 # in units of pixels
im_blur = skimage.filters.gaussian(im_float, sigma=blur_radius)
im_sub = im_float - im_blur

Let's show the original image, the blurred image, and the background subtracted image side-by-side.

In [ ]:
fix, ax = plt.subplots(1,3, figsize=(12,6))
ax[0].imshow(im_float)
ax[1].imshow(im_blur)
ax[2].imshow(im_sub)

Much better. Let's do the rethresholding dance one last time.

In [ ]:
_, (ax1, ax2) = plt.subplots(2, 1, figsize=(9,15))
# plot histogram of normalized intensities
ax1.hist(im_sub.flatten(), bins=300)
ax1.vlines(-0.21, 1e0, 3e3)
ax1.set_xlabel('normalized pixel count')
ax1.set_ylabel('counts')
ax1.set_yscale('log')
# choose threshold and plot image
im_sub_thresh = im_sub < -0.21
ax2.imshow(im_sub_thresh, cmap=plt.cm.Greys_r)

Not too shabby, but we are still picking up some garbage in the
background. We could remove this by only selecting cells that meet a certain
area threshold, but how can we compute the area of each cell? Right now, all
pixels that are deemed to be "bacterial" are labeled as 1.0. To the computer
these are all the same object. We can individually label our segmented cells
by using the `skimage.measure.label` function, which will individually label
islands of pixels. In other words, every connected island of pixels will be assigned a unique value in the returned image.

In [ ]:
im_lab, num_obj = skimage.measure.label(im_sub_thresh, return_num=True)
print('We segmented ' + str(num_obj) + ' objects.')

That is definitely more objects than I want to count by eye! But wait, let's see what the labeled image looks like before continuing:

In [ ]:
plt.imshow(im_lab, cmap=plt.cm.spectral_r)
plt.colorbar()

Looking at the above image, we see that there are far more segmented objects
than there are actual cells. This is because we are segmenting some of the
pixels in the background of the images. 

Our next task will be to separate the actual cells from the imposters. We imagined that we could get rid of these pixels by slecting objects wihch meet a set of area bounds. Before we apply any bounds, let's just look at the areas of all of the cells in our image. It will be easier to work in physical units (microns) rather than arbitrarily sized pixels, so we will convert using the known interpixel spacing. We will use the enormously powerful `regionprops` function to compute areas of every segmented object, then loop through again and remove any that fail our area screen. (See the docs for more on `regionprops`).

In [ ]:
# The interpixel distance
ip_dist = 0.16 # units of um/pixel
# Define a list to store areas of (putative) cells
area = []
# props is an iterable object. It stores, among many things, area of each object.
props = skimage.measure.regionprops(im_lab)
# Loop through and extract areas of objects
for labeled_obj in props:
    area.append(labeled_obj.area * ip_dist**2)

Let's plot the distribution of cell areas.

In [ ]:
# make a histogram of cell areas
plt.hist(area, bins=int(np.sqrt(num_obj)))
plt.xlabel('object area (sq. micron)')
plt.ylabel('counts')
plt.yscale('log')

What would some good bounds
be? Our rule-of-thumb is that *E. coli* is about 2 microns long by one-half to one micron wide. If we approximate our cell as a rectangle, this gets us to an area of
1 or 2 sq micron. Of course, not all of our cells are ideal. We see in our histogram that we have some distribution between  about 1.5 - 3.5 square micron. There is another distribution of junk that is much smaller than our bounds. Let's filter out those objects and see what we catch.

The logic here is tricky. First we create a blank image sized like our original. Then, for each putative cell, compute its area (in $\mu$m$^2$). Then ask if its area is between our cutoffs. If so, find all pixels in the original (*labeled*) image that match our current objects label, and set those pixels to 1 in the new image (we must use `+=` rather than `=` so we don't disturb any other pixels in the new image, as using `=` would).

In [ ]:
approved_obj = np.zeros_like(im_lab)
for labeled_obj in props:
    obj_area = labeled_obj.area * ip_dist**2
    if (obj_area > 1.0) & (obj_area < 4):
        approved_obj += (im_lab == labeled_obj.label)

In [ ]:
# Plot the area-screened image and see how we did
_, ax = plt.subplots(figsize=(10,10))
ax.imshow(approved_obj, cmap=plt.cm.Greys_r)